In [1]:
import pandas as pd
df = pd.read_csv("E:\\odoo project\\rag\\learn-rag\\data.csv")
df = df[df['description'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)


700

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_books",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

C:\Users\Dell\AppData\Local\Temp\ipykernel_6316\3835417732.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [6]:
# vectorize!
qdrant.upload_points(
    collection_name="top_books",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["description"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [7]:
user_prompt = "Suggest me an amazing Fictional Book from sidney sheldon"

In [8]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_books",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'isbn13': 9780385333504, 'isbn10': '0385333501', 'title': 'Welcome to the Monkey House', 'subtitle': 'A Collection of Short Works', 'authors': 'Kurt Vonnegut', 'categories': 'Fiction', 'thumbnail': 'http://books.google.com/books/content?id=vkj0QEE0uRUC&printsec=frontcover&img=1&zoom=1&source=gbs_api', 'description': "Tender stories of love, incisive essays on human greed and misery, and imaginative tales of futuristic happenings reveal Vonnegut's versatility and vision", 'published_year': 1968.0, 'average_rating': 4.14, 'num_pages': 331.0, 'ratings_count': 44749.0} score: 0.47379913093067877
{'isbn13': 9780316919890, 'isbn10': '0316919896', 'title': "A Supposedly Fun Thing I'll Never Do Again", 'subtitle': 'Essays and Arguments Tag - Author of Infinite Jest', 'authors': 'David Foster Wallace', 'categories': 'Humor', 'thumbnail': 'http://books.google.com/books/content?id=QkhpvgEACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api', 'description': 'In this exuberantly praised book - a 

In [9]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [10]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a book specialist. Your top priority is to help guide users into selecting amazing books and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing scientific from as per rating of top 5"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

NotFoundError: <!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>404 Not Found</title>
</head><body>
<h1>Not Found</h1>
<p>The requested URL was not found on this server.</p>
</body></html>